In [1]:
from google.colab import files

uploades = files.upload()

ModuleNotFoundError: No module named 'google'

In [2]:
import scipy

import scipy.io

# Load the MATLAB file

mat_data = scipy.io.loadmat('HW3Data.mat')

# Access variables in the MATLAB file
# For example, if you have a variable 'data' in the MATLAB file:
# data = mat_data['data']
# Now 'data' in Python contains the data from the 'data' variable in the MATLAB file.

Vocabulary = mat_data['Vocabulary']
XTrain = mat_data['XTrain']
XTest = mat_data['XTest']
yTrain = mat_data['yTrain']
yTest = mat_data['yTest']
XTrainSmall = mat_data['XTrainSmall']
yTrainSmall = mat_data['yTrainSmall']

XTrain = XTrain.toarray()
XTest = XTest.toarray()
XTrainSmall = XTrainSmall.toarray()

In [4]:
import numpy as np

def logProd(x):
    # x is vector of logs of probabilities
    # logprod returns the log of the products of the probabilites mentioned above
    log_product = np.sum(x)      # TODO: update definition
    return log_product


In [5]:

import numpy as np

def NB_XGivenY(XTrain, yTrain):
    # XTrain: n x V matrix
    # yTrain: n x 1 vector
    V = XTrain.shape[1]
    D = np.zeros((2, V))
    for y in [1, 2]:
        docs_in_class = XTrain[yTrain[:, 0] == y]
        D[y-1, :] = (np.sum(docs_in_class, axis=0) + 2) / (docs_in_class.shape[0] + 3)
    return D


def NB_YPrior(yTrain):
    num_economist = np.sum(yTrain == 1)
    num_docs = len(yTrain)
    return num_economist / num_docs



In [6]:
import numpy as np


def NB_Classify(D, p, X):
    log_D = np.log(D)
    log_1_minus_D = np.log(1 - D)
    log_prior = np.log([p, 1-p])
    
    yHat = np.zeros(X.shape[0], dtype=int)
    for i in range(X.shape[0]):
        log_p_economist = np.sum(log_D[0, :] * X[i, :] + log_1_minus_D[0, :] * (1 - X[i, :])) + log_prior[0]
        log_p_onion = np.sum(log_D[1, :] * X[i, :] + log_1_minus_D[1, :] * (1 - X[i, :])) + log_prior[1]
        yHat[i] = 1 if log_p_economist > log_p_onion else 2
    return yHat


In [7]:
import numpy as np

def ClassificationError(yHat, yTruth):
    return np.mean(yHat != yTruth)


In [8]:
D = NB_XGivenY(XTrain, yTrain)
p = NB_YPrior(yTrain)


In [9]:
yHatTrain = NB_Classify(D, p, XTrain)
yHatTest = NB_Classify(D, p, XTest)

In [10]:

trainError = ClassificationError(yHatTrain, yTrain)
testError = ClassificationError(yHatTest, yTest)

In [11]:
print("Training Error:", trainError)
print("Testing Error:", testError)

Training Error: 0.3102259215219976
Testing Error: 0.298359096313912


In [12]:
yHatTest = NB_Classify(D, p, XTest)
trainError = ClassificationError(yHatTrain, yTrainSmall)
testError = ClassificationError(yHatTest, yTest)
print(trainError)
print(testError)


0.3102259215219976
0.298359096313912


In [14]:
D_small = NB_XGivenY(XTrainSmall, yTrainSmall)
p_small = NB_YPrior(yTrainSmall)

yHatTrainSmall = NB_Classify(D_small, p_small, XTrainSmall)
yHatTestSmall = NB_Classify(D_small, p_small, XTest)

trainErrorSmall = ClassificationError(yHatTrainSmall, yTrainSmall)
testErrorSmall = ClassificationError(yHatTestSmall, yTest)

print("Training Error (Small):", trainErrorSmall)
print("Testing Error (Small):", testErrorSmall)


Training Error (Small): 0.25862068965517243
Testing Error (Small): 0.2896551724137931


##### With the smaller training set, the training error decreases slightly (from 0.31 to 0.26) due to potential overfitting, where the model may memorize the smaller dataset more easily. However, the test error remains stable at 0.29 for both the full and small datasets. This indicates that despite having less data, the classifier's performance on unseen data doesn't significantly worsen. The prior has a greater impact with less training data, making the model rely more on prior assumptions, but overall, the model's generalization ability remains largely unaffected.


In [15]:
def interpret_words(D, Vocabulary):
     
    top_5_class1 = np.argsort(D[0, :])[-5:]  # For class 1 (The Economist)
    top_5_class2 = np.argsort(D[1, :])[-5:]  # For class 2 (The Onion)
    
    print("Top 5 words for The Economist:")
    for idx in top_5_class1:
        print(Vocabulary[idx, 0])

    print("\nTop 5 words for The Onion:")
    for idx in top_5_class2:
        print(Vocabulary[idx, 0])

     
    ratio_class1 = D[0, :] / D[1, :]
    ratio_class2 = D[1, :] / D[0, :]
    
    top_5_discriminative_class1 = np.argsort(ratio_class1)[-5:]
    top_5_discriminative_class2 = np.argsort(ratio_class2)[-5:]

    print("\nTop 5 discriminative words for The Economist:")
    for idx in top_5_discriminative_class1:
        print(Vocabulary[idx, 0])

    print("\nTop 5 discriminative words for The Onion:")
    for idx in top_5_discriminative_class2:
        print(Vocabulary[idx, 0])


interpret_words(D, Vocabulary)


Top 5 words for The Economist:
['a']
['of']
['in']
['the']
['to']

Top 5 words for The Onion:
['to']
['in']
['a']
['said']
['of']

Top 5 discriminative words for The Economist:
['parliament']
['centr']
['favour']
['reckon']
['organis']

Top 5 discriminative words for The Onion:
['tuesday']
['percent']
['monday']
['5enlarg']
['4enlarg']


##### The discriminative words better describe the two classes because they reflect specific content related to each magazine. The Economist features formal words like "parliament" and "favour," while The Onion uses casual terms like "tuesday" and "percent," which align with its satirical style. In contrast, the common top 5 words are mostly stop words like "a" and "the," which don't offer much insight into the differences between the two publications.